In [3]:
import yfinance as yf
import pandas as pd
import requests
import numpy as np

# Question 1: What's the total sum ($m) of 2023 filings that happenned of Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

filings_dfs = pd.read_html(response.text)

/tmp/ipykernel_11775/4163700054.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filings_dfs = pd.read_html(response.text)


In [5]:
filings = filings_dfs[0]
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [6]:
filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [7]:
filings['Filing Date'] = pd.to_datetime(filings['Filing Date'], format='%b %d, %Y')

In [8]:
filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-


In [9]:
filings.loc[filings['Shares Offered'].astype(str).str.find('-') >= 0, 'Shares Offered'] = np.nan
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'])

In [10]:
def get_avg_price(input_string):
    if input_string == '-':
        price = np.nan
    elif len(input_string.split('-')) == 2:
        lower, higher = input_string.split('-')
        if '$' in lower:
            lower = lower.split('$')[1]
        if '$' in higher:
            higher = higher.split('$')[1]
        price = (float(lower) + float(higher)) / 2
    elif input_string.startswith('$'):
        price = float(input_string.split('$')[1])
    else:
        price = 'ERROR'
    return price

In [11]:
(filings['Price Range'].apply(lambda x: get_avg_price(x)) == 'ERROR').sum()

0

In [12]:
filings['Avg_price'] = filings['Price Range'].apply(lambda x: get_avg_price(x))

In [13]:
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  252 non-null    float64       
 5   Avg_price       258 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 15.4+ KB


In [14]:
filings['Shares_offered_value'] = filings['Shares Offered'] * filings['Avg_price']

In [15]:
filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN


In [16]:
filings.loc[(filings['Filing Date'].dt.year == 2023) & (filings['Filing Date'].dt.dayofweek == 4), 'Shares_offered_value'].shape

(32,)

In [22]:
filings.loc[(filings['Filing Date'].dt.year == 2023) & (filings['Filing Date'].dt.dayofweek == 4), 'Shares_offered_value'].notna().sum()

25

In [23]:
filings.loc[(filings['Filing Date'].dt.year == 2023) & (filings['Filing Date'].dt.dayofweek == 4), 'Shares_offered_value'].sum()/1000000

285.7

286 millon $

# Question 2: Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 185 tickers. Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Addtional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [24]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


/tmp/ipykernel_11775/2153473391.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [25]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


/tmp/ipykernel_11775/439280575.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [26]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [27]:
stacked_ipos_df.loc[stacked_ipos_df['IPO Date'] < '2024-03-01', 'Symbol'].shape

(185,)

In [28]:
tickers = stacked_ipos_df.loc[stacked_ipos_df['IPO Date'] < '2024-03-01', 'Symbol'].to_list()

In [29]:
tickers.sort()

In [30]:
ticker_mapping = {
    'PTHR': 'PTHRU', 
    'RYZB': 'RYZB'   
}

In [31]:
tickers = ['PTHRU' if t == 'PTHR' else t for t in tickers]

In [32]:
tickers_string = ' '.join(tickers).replace('PTHR', 'PTHRU')
# RayzeBio, Inc. does not seem to be traded anymore

In [33]:
data = yf.download(tickers_string, start="2023-01-01")

[*********************100%%**********************]  185 of 185 completed

2 Failed downloads:
['PTHRUU', 'RYZB']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [34]:
stacked_ipos_df.loc[stacked_ipos_df['IPO Date'] < '2024-03-01']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
34,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.80,163.63%
35,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.14,-96.86%
36,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.15,1.60%
37,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.77,-69.38%
38,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.05,-74.50%
...,...,...,...,...,...,...
214,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.16,-74.48%
215,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.50,-90.40%
216,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.59,150.73%
217,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [35]:
stacked_ipos_df[stacked_ipos_df['Symbol'] == 'CHSN']['IPO Date']

179   2023-03-30
Name: IPO Date, dtype: datetime64[ns]

In [36]:
growth = {}
for ticker in tickers:
    try:
        ipo_date = data[data['Adj Close'][ticker].notna()].index[0]
        ipo_idx = data.index.get_loc(ipo_date)
        ipo_price = data['Adj Close'][ticker].iloc[ipo_idx]
        growth[ticker] = [(data['Adj Close'][ticker].iloc[i]-ipo_price)/ipo_price for i in range(ipo_idx, ipo_idx+30)]
    except:
        print(ticker)

df_growth = pd.DataFrame(growth)

DYCQ
LEGT
PTHRU
RYZB


In [37]:
df_growth.T.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,181.0,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,...,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000
mean,0.0,-0.053568,-0.061671,-0.069175,-0.079447,-0.087056,-0.093864,-0.104689,-0.105880,-0.103358,...,-0.085163,-0.081967,-0.088754,-0.090860,-0.093606,-0.100961,-0.101141,-0.070312,-0.065560,-0.065911
std,0.0,0.172889,0.206936,0.249107,0.265115,0.295587,0.310356,0.299981,0.322735,0.354094,...,0.493599,0.512867,0.473753,0.480087,0.485429,0.481636,0.522515,0.784932,0.786188,0.784360
min,0.0,-0.846431,-0.891267,-0.913359,-0.905743,-0.918882,-0.912323,-0.914739,-0.909877,-0.904384,...,-0.950121,-0.951674,-0.951674,-0.950639,-0.951674,-0.951674,-0.954781,-0.955471,-0.953055,-0.957197
25%,0.0,-0.095890,-0.122449,-0.150000,-0.167500,-0.219455,-0.250000,-0.218750,-0.253429,-0.271429,...,-0.347917,-0.347352,-0.368580,-0.380645,-0.365639,-0.382879,-0.400000,-0.394813,-0.360784,-0.372277
50%,0.0,0.000000,-0.000899,-0.002314,-0.004988,-0.005714,-0.029595,-0.026849,-0.019324,-0.017448,...,-0.025000,-0.022222,-0.007500,-0.015491,-0.016489,-0.027157,-0.013793,-0.011818,-0.024537,-0.020654
75%,0.0,0.016667,0.021407,0.011162,0.010204,0.010204,0.007857,0.006897,0.008815,0.011896,...,0.015584,0.014706,0.038710,0.029362,0.040847,0.030000,0.040647,0.046142,0.045305,0.030118
max,0.0,0.362069,0.464015,1.380000,1.083710,1.262443,1.529870,1.173913,1.359740,1.751948,...,2.970454,3.500000,2.871041,2.846154,2.803394,2.427273,3.817886,8.056122,8.081632,8.265306


In [43]:
df_growth.T.describe().loc['75%']

0     0.000000
1     0.016667
2     0.021407
3     0.011162
4     0.010204
5     0.010204
6     0.007857
7     0.006897
8     0.008815
9     0.011896
10    0.012069
11    0.014377
12    0.017699
13    0.020355
14    0.020000
15    0.019900
16    0.019608
17    0.012745
18    0.015929
19    0.021154
20    0.015584
21    0.014706
22    0.038710
23    0.029362
24    0.040847
25    0.030000
26    0.040647
27    0.046142
28    0.045305
29    0.030118
Name: 75%, dtype: float64

After 28 days.

# Question 3: Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

In [44]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [45]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [46]:
len(set(LARGE_STOCKS))

33

In [47]:
largest = yf.download(LARGEST_STOCKS, start="2013-12-15", end="2023-12-31")
large = yf.download(LARGE_STOCKS, start="2013-12-15", end="2023-12-31")

[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  33 of 33 completed


In [48]:
largest = largest['Adj Close']
large = large['Adj Close']

In [49]:
large.shape

(2606, 33)

In [50]:
largest[[f'{t}_growth_7d' for t in LARGEST_STOCKS]] = largest / largest.shift(7)
large[[f'{t}_growth_7d' for t in LARGE_STOCKS]] = large / large.shift(7)

In [51]:
largest = largest.loc['2014-01-01':][[c for c in largest.columns if 'growth' in c]].mean(axis=1)

In [52]:
large = large.loc['2014-01-01':][[c for c in large.columns if 'growth' in c]].mean(axis=1)

In [53]:
(large.index == largest.index).all()

True

In [54]:
large.shape

(2595,)

In [55]:
(large > largest).sum()/large.shape[0]

0.4662813102119461

47%

# Question 4: What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

In [56]:
df = pd.read_parquet('/home/tim/Downloads/stocks_df_combined_trunc_2014_2023.parquet.brotli')

In [57]:
profit = 0
trades = 0
for date in df[df['Weekday'] == 4]['Date'].unique():
    if not df[(df['Date'] == date) & (df['cci'] > 200)].empty:
        for ticker in df[(df['Date'] == date) & (df['cci'] > 200)]['Ticker']:
            trades += 1
            buy_price = df[(df['Date'] == date) & (df['Ticker'] == ticker)]['Adj Close_x'].values[0]
            for extra_days in [0, 1, 2, 3]:
                if not df[(df['Date'] == date + pd.Timedelta(days=7+ extra_days)) & (df['Ticker'] == ticker)]['Adj Close_x'].empty:
                    sell_price = df[(df['Date'] == date + pd.Timedelta(days=7+ extra_days)) & (df['Ticker'] == ticker)]['Adj Close_x'].values[0]
                    break
            profit += ((sell_price - buy_price)/buy_price) * 1000

In [58]:
trades

460

In [61]:
profit

1206.1765251011268

1k

#  Question 5: Finding Your Strategy for IPOs

I would look at the companies profit in comparison to the profit of companies in the same sector that are already listed. In addition, it would be good to look at the companies evaluation in comparison to companies in the same sector that are already listed. If these indicators show that we might get more profit per share than what is currently available we can invest.